#### Import necessary libaries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pywt
from scipy.signal import find_peaks, peak_prominences
from scipy.signal import argrelextrema
import os
from pandas import ExcelWriter

#### Assign directories

In [2]:
directory_ERGCamp6 = 'Data/ER-GCamp6'
directory_TED = 'Data/TED'

#### Determines the most frequent timepoint of the LED peak.  The application time is 10 frames after the LED peak. Requiere the appoximate time frame of the LED signal.

In [3]:
def get_application_time(data, led_peak_earliest, led_peak_latest, lag=10):
    # get highest peak per cell in video between earliest and latest
    LED_max = data.iloc[led_peak_earliest:led_peak_latest,:].idxmax()
    
    # get most common peak position in video
    maxi = np.bincount(LED_max).argmax()
    
    # add n=lag frames for actual application time
    return maxi, maxi + lag

#### Wavelet transform of one trace (numpy array)

In [4]:
def wavelet_transform(signal, wavelet="coif3"):

    ca5, cd5, cd4, cd3, cd2, cd1 = pywt.wavedec(signal, wavelet, level=5)

    def thresh(x):
        # calculate thresh:
        # https://www.hindawi.com/journals/mpe/2016/3195492/
        threshold = np.std(x) * np.sqrt(2*np.log(len(x)))
        return pywt.threshold(x, threshold)

    cd5 = thresh(cd5)
    cd4 = thresh(cd4)*0
    cd3 = thresh(cd3)*0
    cd2 = thresh(cd2)*0
    cd1 = thresh(cd1)*0

    return pywt.waverec([ca5, cd5, cd4, cd3, cd2, cd1], wavelet)

#### Determine threshold for inflection point

In [5]:
def get_threshold_inflection(derivative, application_time):
    #lower peak detection of the dericative
    peaks_pre, peak_heights_pre = find_peaks(-derivative, prominence=0.01)
    
    #counts peaks that appear before the application of the agonist and cannot result from the agonist
    count = 0
    for i in peaks_pre:
        if i < application_time:
            count += 1
    # sets the threshold 0.001 bigger than that of the peaks that appeared before the application        
    if count > 0:
        threshold = np.max(peak_heights_pre['prominences'][:count]) + 0.001
    else:
        threshold = 0.01
    
    return threshold

#### Determines if cell (one ROI) shows reaction. If so, timepoint of first peak, number of peaks, slope and relative peak height is calculated, else 'no reaction'. Requieres data of one cell and application time.

In [6]:
def get_results_roi(cell_data, cell_data_ori, application_time, LED_max):
    signal = wavelet_transform(cell_data)
    extrema = argrelextrema(np.array(signal), np.less)
    
    dy1 = np.diff(signal)    
    
    threshold = get_threshold_inflection(dy1, application_time)
    
    peaks, peak_heights = find_peaks(-dy1, prominence=threshold)
    
    #get rid of peaks that are not clear oscillations
    peaks_osci = [peak for peak in peaks if dy1[peak] <= -0.01]
    
    #count oscillation peaks in defined time window, because not all videos have the same length 
    peak_number_thresh = application_time + 700
    number_peaks = 0
    for i in peaks_osci:
        if i <= peak_number_thresh:
            number_peaks += 1
    
    #reaction when more than one peak is detected
    if len(peaks) > 0:
        #get first inflection point
        first_peak = peaks[0]
        
        #get first low extreme point after infection point
        for i in extrema[0]:
            if i > peaks[0]: 
                extrema_first = i
                break
        
        #calculate slope
        x = [first_peak-1, first_peak+1]
        y = [signal[first_peak-1], signal[first_peak+1]]
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        
        # calculate relative peak height/signal strength
        rel_peak_height = signal[application_time] - signal[extrema_first]
        
        return first_peak, number_peaks, slope, rel_peak_height
     
    else: 
        return 'no reaction'   

#### Get results of one csv file

In [7]:
def get_results_csv(file):
    rois_pre = pd.read_csv(file, sep = '\t', index_col=0)
    rois = rois_pre.drop(labels = ["Average", "Err"], axis=1)
    
    #approximate time window of the LED appearance
    if len(rois)<=1000:
        led_peak_earliest=200
        led_peak_latest=300
    else:
        led_peak_earliest=450
        led_peak_latest=600
    
    LED_max, application_time = get_application_time(rois, led_peak_earliest, led_peak_latest)
    
    #get indices right
    LED_max = LED_max-1
    application_time = application_time-1
    
    # get rid of LED peak for the wavelet transformation because it is artificial
    rois_noLED = rois.copy()
    LED_min = LED_max - 5
    rois_noLED.iloc[LED_max,:] = rois.iloc[LED_min:LED_max,:].mean(axis=0)
    
    # iterate over rois in csv to get results into one dataframe
    reaction_times = []
    number_of_peaks = []
    slopes = []
    rel_peak_heights = []
    for i in rois:
        results = get_results_roi(rois_noLED[i], rois[i], application_time, LED_max)
        if results == "no reaction":
            number_peaks = 0
            reaction_times.append(np.nan)
            number_of_peaks.append(number_peaks)
            slopes.append(np.nan)
            rel_peak_heights.append(np.nan)
        else:
            first_peak, number_peaks, slope, rel_peak_height = results
            reaction_time = (first_peak - application_time)*100
            reaction_times.append(reaction_time)
            number_of_peaks.append(number_peaks)
            slopes.append(slope)
            rel_peak_heights.append(rel_peak_height)
    return reaction_times, number_of_peaks, slopes, rel_peak_heights

#### Apply processing on all csv files in directory and merge all results into a table

In [8]:
def get_results_all(directory):
    Reaction_times = []
    Number_of_peaks = [] 
    Slopes = []
    Rel_peak_heights = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            reaction_times, number_of_peaks, slopes, rel_peak_heights = get_results_csv(os.path.join(directory, filename))
            filenames.append(filename[0:-4])
            Reaction_times.extend(reaction_times)
            Number_of_peaks.extend(number_of_peaks)
            Slopes.extend(slopes)
            Rel_peak_heights.extend(rel_peak_heights)       
            result_table = pd.DataFrame( pd.DataFrame(
                {'Reaction_times': Reaction_times,
                 'Number_of_peaks': Number_of_peaks,
                 'Slopes': Slopes,
                 'Signal_strength':Rel_peak_heights}))
        else:
            continue
    return result_table

#### Apply pipeline

In [9]:
ERGCamp6 = get_results_all(directory_ERGCamp6)
TED = get_results_all(directory_TED)

#### Save result table as an excel file

In [10]:
ERGCamp6.to_excel("ERGCamp6_Results.xlsx")
TED.to_excel("TED_Results.xlsx")

PermissionError: [Errno 13] Permission denied: 'ERGCamp6_Results.xlsx'